In [1]:
import torch
import os

In [2]:
torch.ones(1).cuda()

tensor([1.], device='cuda:0')

In [5]:
!conda install pytorch torchvision cudatoolkit=10.2 -c pytorch -y

Fetching package metadata ...

In [4]:
!ls /home/itdfh/data/dfdc-subset/train_spectrograms_part-5/real/ | head -10

ls: cannot access /home/itdfh/data/dfdc-subset/train_spectrograms_part-5/real/: No such file or directory


In [5]:
def get_feats(directory, nfirst=25, feats=700):
    vidpaths = sorted([os.path.join(directory, p) for p in os.listdir(directory)])

    samples = []
    for vp in vidpaths:
        samples.extend([os.path.join(vp, p) for p in sorted(os.listdir(vp)) if p[-5:] == '24.pt'])
        
    samples = samples[:nfirst]
    return torch.stack([torch.load(sample, map_location=torch.device('cpu'))[:,:feats] for sample in samples])

In [6]:
data_spec = get_feats('/home/itdfh/data/dfdc-subset/train_spectrograms_part-5/fake/')
data_spec.shape

torch.Size([25, 24, 700])

In [7]:
data_xcep = get_feats('/home/itdfh/data/dfdc-subset/train_xception_part-5/fake/', feats=2048)
data_xcep.shape

torch.Size([25, 24, 2048])

In [8]:
data = torch.cat([data_xcep, data_spec], dim=-1)
data.shape

torch.Size([25, 24, 2748])

In [9]:
# import matplotlib.pyplot as plt

# # t = t.numpy()
# plt.imshow(t.transpose(), aspect='auto', origin='bottom', cmap='jet')

In [10]:
data.cuda()

tensor([[[ 3.4736e-01,  2.3120e-01,  4.2913e-02,  ..., -1.3379e+00,
          -1.7537e+00, -1.4854e+00],
         [ 1.4201e-01,  1.3520e-01,  3.1020e-01,  ..., -1.4600e+00,
          -1.3409e+00, -1.4203e+00],
         [ 8.8936e-02,  3.6510e-01,  1.4192e-01,  ..., -1.5401e+00,
          -1.3727e+00, -1.8840e+00],
         ...,
         [ 2.7180e-01,  1.3079e-01,  1.6493e-01,  ..., -1.2839e+00,
          -1.4657e+00, -1.3755e+00],
         [ 2.8031e-01,  2.5773e-01,  1.5841e-01,  ..., -1.2791e+00,
          -1.4905e+00, -1.5813e+00],
         [ 1.6904e-01,  4.4492e-01,  2.5560e-01,  ..., -1.7823e+00,
          -1.5875e+00, -1.5905e+00]],

        [[ 2.5960e-01,  1.8374e-02,  1.1157e-03,  ..., -1.5968e+00,
          -1.3002e+00, -1.4684e+00],
         [ 1.7712e-01,  2.2126e-01,  7.8873e-02,  ..., -1.2426e+00,
          -1.3833e+00, -1.5212e+00],
         [ 1.0899e-01,  1.3758e-01,  1.9101e-01,  ..., -1.4018e+00,
          -1.2912e+00, -1.8698e+00],
         ...,
         [ 1.1210e-01,  1

In [2]:
from torch import nn

class FrimageNet(nn.Module):
    def __init__(self, feature_size, num_layers=2, num_hidden_nodes=512, device='cuda'):
        super(FrimageNet, self).__init__()
        self.device = device
        self.num_layers = num_layers
        self.num_hidden_nodes = num_hidden_nodes

        # input dim is 167, output 200
        self.lstm = nn.LSTM(feature_size, num_hidden_nodes,
                            batch_first=True, num_layers=num_layers)
        # fully connected
        self.fc1 = nn.Linear(num_hidden_nodes, num_hidden_nodes)
        self.act = nn.Sigmoid()
        self.fc2 = nn.Linear(num_hidden_nodes, 2)
        self.softmax = nn.Softmax()

    def forward(self, x, hidden):
#         print(x.device, hidden[0].device)
        y, hidden = self.lstm(x, hidden)    # returns the two outputs
        y = y[:, -1, :]  # get only the last output
        y = self.fc1(y)
        y = self.fc2(y)
        y = F.softmax(y, dim=1)

        return y, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).datac
        hidden = (weight.new(self.num_layers, batch_size, self.num_hidden_nodes).zero_().to(self.device),
                  weight.new(self.num_layers, batch_size, self.num_hidden_nodes).zero_().to(self.device))
        return hidden


In [ ]:
net = FrimageNet(feature_size=100)
net = net.cuda()

In [ ]:
h = net.init_hidden(25)
data = data.cuda()
net(data, h)

In [ ]:
torch.ones(5).cuda()